In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import streamlit as st
from PIL import Image

DATASET_PATH = "/path/to/your/dataset"
CATEGORIES = ["flood", "earthquake", "wildfire", "hurricane"]
IMG_SIZE = 224

def load_and_preprocess_images():
    images, labels = [], []
    for label, category in enumerate(CATEGORIES):
        folder = os.path.join(DATASET_PATH, category)
        for filename in os.listdir(folder):
            img_path = os.path.join(folder, filename)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = img / 255.0
            images.append(img)
            labels.append(label)
    images = np.array(images)
    labels = np.array(labels)
    labels = to_categorical(labels, num_classes=len(CATEGORIES))
    return images, labels

images, labels = load_and_preprocess_images()
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
output_layer = Dense(len(CATEGORIES), activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)
model.save("disaster_classification_model.h5")

for layer in base_model.layers[-30:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5), loss="categorical_crossentropy", metrics=["accuracy"])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

history_finetune = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32, callbacks=[early_stopping])
model.save("fine_tuned_disaster_model.h5")

MODEL_PATH = "fine_tuned_disaster_model.h5"
model = tf.keras.models.load_model(MODEL_PATH)

def preprocess_image(image):
    img = np.array(image)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    return img

st.title("🌍 Disaster Detection from Satellite Images")
st.write("Upload a satellite image to detect the type of disaster.")

uploaded_file = st.file_uploader("Choose a satellite image...", type=["jpg", "png", "jpeg"])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption="Uploaded Image", use_column_width=True)
    processed_image = preprocess_image(image)
    prediction = model.predict(processed_image)
    predicted_class = CATEGORIES[np.argmax(prediction)]
    confidence = np.max(prediction) * 100

    st.subheader(f"🛑 Predicted Disaster: **{predicted_class}**")
    st.write(f"📊 Confidence: **{confidence:.2f}%**")

    if predicted_class == "Flood":
        st.write("🌊 Flood detected! Evacuation plans should focus on high ground areas.")
    elif predicted_class == "Earthquake":
        st.write("🌍 Earthquake detected! Ensure buildings are earthquake-resistant.")
    elif predicted_class == "Wildfire":
        st.write("🔥 Wildfire detected! Monitor wind direction and evacuate accordingly.")
    elif predicted_class == "Hurricane":
        st.write("🌪 Hurricane detected! Seek shelter in safe zones.")
